# PySpark and Big Data Project

Analyzing Common Crawl Data with RDDs

Initialize a new Spark Context and read in the domain graph as an RDD.

In [23]:
# Import required modules
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession.builder.getOrCreate()


# Get SparkContext
sc = spark.sparkContext

spark.sparkContext.setLogLevel("ERROR")


In [24]:
# Read Domains CSV File into an RDD
file_path = 'data/cc-main-limited-domains.csv'
common_crawl_domain_counts = sc.textFile(file_path)

# Display first few domains from the RDD
common_crawl_domain_counts.take(5)

['367855\t172-in-addr\tarpa\t1',
 '367856\taddr\tarpa\t1',
 '367857\tamphic\tarpa\t1',
 '367858\tbeta\tarpa\t1',
 '367859\tcallic\tarpa\t1']

Applying `fmt_domain_graph_entry` over `common_crawl_domain_counts` and saveing the result as a new RDD named `formatted_host_counts`.

In [25]:
def fmt_domain_graph_entry(entry):
    """
    Formats a Common Crawl domain graph entry. Extracts the site_id, 
    top-level domain (tld), domain name, and subdomain count as seperate items.
    """

    # Split the entry on delimiter ('\t') into site_id, domain, tld, and num_subdomains
    site_id, domain, tld, num_subdomains = entry.split('\t')        
    return int(site_id), domain, tld, int(num_subdomains)

# Apply `fmt_domain_graph_entry` to the raw data RDD
formatted_host_counts = common_crawl_domain_counts.map(fmt_domain_graph_entry)

# Display the first few entries of the new RDD
formatted_host_counts.take(10)

[(367855, '172-in-addr', 'arpa', 1),
 (367856, 'addr', 'arpa', 1),
 (367857, 'amphic', 'arpa', 1),
 (367858, 'beta', 'arpa', 1),
 (367859, 'callic', 'arpa', 1),
 (367860, 'ch', 'arpa', 1),
 (367861, 'd', 'arpa', 1),
 (367862, 'home', 'arpa', 7),
 (367863, 'iana', 'arpa', 1),
 (367907, 'local', 'arpa', 1)]

Applying `extract_subdomain_counts` over `common_crawl_domain_counts` and saving the result as a new RDD named `host_counts`.

In [26]:
def extract_subdomain_counts(entry):
    """
    Extract the subdomain count from a Common Crawl domain graph entry.
    """
    
    # Split the entry on delimiter ('\t') into site_id, domain, tld, and num_subdomains
    site_id, domain, tld, num_subdomains = entry.split('\t')
    
    # return ONLY the num_subdomains
    return int(num_subdomains)


# Apply `extract_subdomain_counts` to the raw data RDD
host_counts = common_crawl_domain_counts.map(extract_subdomain_counts)

# Display the first few entries
host_counts.take(10)

[1, 1, 1, 1, 1, 1, 1, 7, 1, 1]

Using `host_counts`, calculate the total number of subdomains across all domains in the dataset, save the result to a variable named `total_host_counts`.

In [27]:
# Reduce the RDD to a single value, the sum of subdomains, with a lambda function as the reduce function
total_host_counts = host_counts.reduce(lambda a,b: a+b)

# Display result count
print(total_host_counts)

595466


Stop the current `SparkSession` and `sparkContext` before moving on to analyze the data with SparkSQL

In [28]:
# Stop the sparkContext and the SparkSession
spark.stop()

## Exploring Domain Counts with PySpark DataFrames and SQL

Create a new `SparkSession` and assign it to a variable named `spark`.

In [29]:
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

# Read the target file into a DataFrame
common_crawl = spark.read\
.option('delimiter', '\t')\
.option('inferSchema', True)\
.csv(file_path)


# Display the DataFrame to the notebook
common_crawl.show(5, truncate = False)

+------+-----------+----+---+
|_c0   |_c1        |_c2 |_c3|
+------+-----------+----+---+
|367855|172-in-addr|arpa|1  |
|367856|addr       |arpa|1  |
|367857|amphic     |arpa|1  |
|367858|beta       |arpa|1  |
|367859|callic     |arpa|1  |
+------+-----------+----+---+
only showing top 5 rows



Rename the DataFrame's columns to the following: 

- site_id
- domain
- top_level_domain
- num_subdomains

In [30]:
# Rename the DataFrame's columns with `withColumnRenamed()`
common_crawl =common_crawl\
.withColumnRenamed("_c0", "site_id")\
.withColumnRenamed("_c1", "domain")\
.withColumnRenamed("_c2", "top_level_domain")\
.withColumnRenamed("_c3", "num_subdomains")\

# Display the first few rows of the DataFrame and the new schema
common_crawl.show(5, truncate = False)


+-------+-----------+----------------+--------------+
|site_id|domain     |top_level_domain|num_subdomains|
+-------+-----------+----------------+--------------+
|367855 |172-in-addr|arpa            |1             |
|367856 |addr       |arpa            |1             |
|367857 |amphic     |arpa            |1             |
|367858 |beta       |arpa            |1             |
|367859 |callic     |arpa            |1             |
+-------+-----------+----------------+--------------+
only showing top 5 rows



## Reading and Writing Datasets to Disk

Saveing, reading and displaying the `common_crawl` DataFrame as parquet files in a directory called `./results/common_crawl/`.

In [31]:
# Save the `common_crawl` DataFrame to a series of parquet files
common_crawl.write.parquet('./results/common_crawl/', mode="overwrite")

# Read from parquet directory
common_crawl_domains = spark.read.parquet('./results/common_crawl/')

# Display the first few rows of the DataFrame and the schema
common_crawl_domains.show(5,truncate=False)

+-------+-----------+----------------+--------------+
|site_id|domain     |top_level_domain|num_subdomains|
+-------+-----------+----------------+--------------+
|367855 |172-in-addr|arpa            |1             |
|367856 |addr       |arpa            |1             |
|367857 |amphic     |arpa            |1             |
|367858 |beta       |arpa            |1             |
|367859 |callic     |arpa            |1             |
+-------+-----------+----------------+--------------+
only showing top 5 rows



## Querying Domain Counts with PySpark DataFrames and SQL

Create a local temporary view from `common_crawl_domains`

In [32]:
# Create a temporary view in the metadata for this `SparkSession`
common_crawl_domains.createOrReplaceTempView("common_crawl_domains")

Calculate the total number of domains for each top-level domain in the dataset, using Dataframe and SQL methods

In [33]:
# Aggregate the DataFrame using DataFrame methods
common_crawl_domains\
.select(["top_level_domain", "num_subdomains"])\
.groupBy("top_level_domain").sum("num_subdomains")\
.orderBy("sum(num_subdomains)", ascending = False)\
.show(truncate = False)

# Aggregate the DataFrame using SQL
query = """ SELECT top_level_domain, sum(num_subdomains)
FROM common_crawl_domains
GROUP BY top_level_domain 
ORDER BY sum(num_subdomains) DESC
"""

spark.sql(query).show(truncate = False)

+----------------+-------------------+
|top_level_domain|sum(num_subdomains)|
+----------------+-------------------+
|edu             |484438             |
|gov             |85354              |
|travel          |10768              |
|coop            |8683               |
|jobs            |6023               |
|post            |143                |
|map             |40                 |
|arpa            |17                 |
+----------------+-------------------+

+----------------+-------------------+
|top_level_domain|sum(num_subdomains)|
+----------------+-------------------+
|edu             |484438             |
|gov             |85354              |
|travel          |10768              |
|coop            |8683               |
|jobs            |6023               |
|post            |143                |
|map             |40                 |
|arpa            |17                 |
+----------------+-------------------+



Calculate the total number of subdomains for each top-level domain in the dataset.

In [34]:
# Aggregate the DataFrame using DataFrame methods
common_crawl_domains\
.select(["top_level_domain","domain", "num_subdomains"])\
.groupBy(["top_level_domain", "domain"]).sum("num_subdomains")\
.orderBy("sum(num_subdomains)", ascending = False)\
.show(truncate = False)

# Aggregate the DataFrame using SQL
query = """ SELECT top_level_domain, domain, sum(num_subdomains)
FROM common_crawl_domains
GROUP BY top_level_domain, domain 
ORDER BY sum(num_subdomains) DESC
"""

spark.sql(query).show(truncate = False)


+----------------+--------+-------------------+
|top_level_domain|domain  |sum(num_subdomains)|
+----------------+--------+-------------------+
|edu             |academia|9657               |
|edu             |mit     |7114               |
|edu             |stanford|7015               |
|edu             |harvard |5497               |
|edu             |wisc    |5376               |
|gov             |nasa    |5370               |
|edu             |tamu    |5088               |
|edu             |berkeley|4874               |
|edu             |umn     |4748               |
|edu             |ncsu    |4448               |
|edu             |cuny    |4260               |
|edu             |unc     |4249               |
|edu             |cornell |4235               |
|edu             |ucla    |4217               |
|edu             |ucsd    |4122               |
|edu             |ucdavis |4105               |
|edu             |umich   |4067               |
|edu             |psu     |3840         

How many sub-domains does `nps.gov` have? Filter the dataset to that website's entry, display the columns `top_level_domain`, `domain`, and `num_subdomains` in your result.

In [35]:
# Filter the DataFrame using DataFrame Methods
common_crawl_domains\
.filter(common_crawl_domains.top_level_domain == "gov")\
.filter(common_crawl_domains.domain == "nps")\
.show(truncate = False)

# Filter the DataFrame using SQL
query = """ SELECT top_level_domain, domain, sum(num_subdomains)
FROM common_crawl_domains
WHERE top_level_domain = 'gov'and domain = 'nps'
GROUP BY top_level_domain, domain 
ORDER BY sum(num_subdomains) DESC
"""

spark.sql(query).show(truncate = False)


+--------+------+----------------+--------------+
|site_id |domain|top_level_domain|num_subdomains|
+--------+------+----------------+--------------+
|57661852|nps   |gov             |178           |
+--------+------+----------------+--------------+

+----------------+------+-------------------+
|top_level_domain|domain|sum(num_subdomains)|
+----------------+------+-------------------+
|gov             |nps   |178                |
+----------------+------+-------------------+



Close the `SparkSession` and underlying `sparkContext`.

In [36]:
# Stop the notebook's `SparkSession` and `sparkContext`
spark.stop()